In [2]:
!pip -q install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 129.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 7.4 MB/s eta 0:00:00


In [6]:
import numpy as np
import matplotlib.pyplot as plt
import gradio as gr

def initialize(Nx, L=1.0, Wall_Velocity=1.0, mu=0.01, rho=1.0, dt=0.001):
    Ny = Nx
    h = L / (Nx - 1)
    x = np.linspace(0, L, Nx)
    y = np.linspace(0, L, Ny)
    Vo = np.zeros((Nx, Ny))
    St = np.zeros_like(Vo)
    u = np.zeros_like(Vo)
    v = np.zeros_like(Vo)
    return Nx, Ny, L, h, x, y, Vo, St, u, v, Wall_Velocity, mu, rho, dt

def apply_boundary_conditions(Vo, St, h, Wall_Velocity):
    Nx, Ny = Vo.shape
    Vo[:, -1] = -2 * St[:, -2] / h**2 - 2 * Wall_Velocity / h
    Vo[:, 0]  = -2 * St[:, 1] / h**2
    Vo[0, :]  = -2 * St[1, :] / h**2
    Vo[-1, :] = -2 * St[-2, :] / h**2
    return Vo

def solve_vorticity_transport(Vo, St, h, mu, rho, dt):
    Vop = Vo.copy()
    Vo[1:-1, 1:-1] = Vop[1:-1, 1:-1] + dt * (
        (- (St[1:-1, 2:] - St[1:-1, :-2]) / (2 * h)) * (Vop[2:, 1:-1] - Vop[:-2, 1:-1]) / (2 * h) +
        (  (St[2:, 1:-1] - St[:-2, 1:-1]) / (2 * h)) * (Vop[1:-1, 2:] - Vop[1:-1, :-2]) / (2 * h) +
        mu / rho * (
            Vop[2:, 1:-1] + Vop[:-2, 1:-1] + Vop[1:-1, 2:] + Vop[1:-1, :-2] - 4 * Vop[1:-1, 1:-1]
        ) / h**2
    )
    return Vo

def solve_stream_function(Vo, St, h):
    St[1:-1, 1:-1] = 0.25 * (Vo[1:-1, 1:-1] * h**2 + St[2:, 1:-1] + St[:-2, 1:-1] + St[1:-1, 2:] + St[1:-1, :-2])
    return St

def compute_velocity(St, u, v, h, Wall_Velocity):
    u[1:-1, 1:-1] = (St[1:-1, 2:] - St[1:-1, :-2]) / (2 * h)
    v[1:-1, 1:-1] = -(St[2:, 1:-1] - St[:-2, 1:-1]) / (2 * h)
    u[:, -1] = Wall_Velocity
    return u, v

def compute_divergence(u, v, h):
    dudx = (u[2:, 1:-1] - u[:-2, 1:-1]) / (2 * h)
    dvdy = (v[1:-1, 2:] - v[1:-1, :-2]) / (2 * h)
    divergence = np.zeros_like(u)
    divergence[1:-1, 1:-1] = dudx + dvdy
    return divergence

def lid_driven_cavity_solver(Nx=64, Wall_Velocity=1.0, mu=0.01, rho=1.0, dt=0.001, maxIt=10000, maxe=1e-7):
    Nx, Ny, L, h, x, y, Vo, St, u, v, Wall_Velocity, mu, rho, dt = initialize(Nx, Wall_Velocity=Wall_Velocity, mu=mu, rho=rho, dt=dt)
    for iter in range(maxIt):
        Vo = apply_boundary_conditions(Vo, St, h, Wall_Velocity)
        Vo_new = solve_vorticity_transport(Vo.copy(), St, h, mu, rho, dt)
        St = solve_stream_function(Vo_new, St, h)
        error = np.max(np.abs(Vo_new - Vo))
        Vo = Vo_new
        if iter > 10 and error < maxe:
            break
    u, v = compute_velocity(St, u, v, h, Wall_Velocity)
    return x, y, u, v, h


def smart_simulate(Nx, known_variable, rho, U, mu, Re, dt, maxIt):
    L = 1.0  # domain length

    # --- Smart Computation Logic ---
    if known_variable == "Density (ρ)":
        rho = Re * mu / (U * L)
    elif known_variable == "Wall Velocity (U)":
        U = Re * mu / (rho * L)
    elif known_variable == "Viscosity (μ)":
        mu = rho * U * L / Re
    elif known_variable == "Reynolds Number (Re)":
        Re = rho * U * L / mu

    x, y, u, v, h = lid_driven_cavity_solver(Nx=int(Nx), Wall_Velocity=U, mu=mu, rho=rho, dt=dt, maxIt=int(maxIt))

    divergence = compute_divergence(u, v, h)
    max_div = np.max(np.abs(divergence))

    X, Y = np.meshgrid(x, y, indexing='ij')

    # Plot U-Velocity
    fig1, ax1 = plt.subplots()
    cf = ax1.contourf(x, y, u.T, 20, cmap='plasma')
    fig1.colorbar(cf)
    ax1.set_title(f"U-Velocity Contour (Re={Re:.1f})")
    ax1.axis('scaled')

    # Centerline Velocity
    fig2, ax2 = plt.subplots()
    ax2.plot(y, u[int(Nx) // 2, :])
    ax2.set_title("Centerline u-velocity")
    ax2.grid()

    # Streamlines
    fig3, ax3 = plt.subplots()
    ax3.streamplot(x, y, u.T, v.T, density=1.3, color='black')
    ax3.set_title("Streamlines")
    ax3.axis('scaled')

    # Divergence Plot
    fig4, ax4 = plt.subplots()
    div_plot = ax4.contourf(x, y, divergence.T, 20, cmap='coolwarm')
    fig4.colorbar(div_plot)
    ax4.set_title(f"Divergence of Velocity Field\nMax |div| = {max_div:.2e}")
    ax4.axis('scaled')

    return fig1, fig2, fig3, fig4, f"ρ = {rho:.4f}, U = {U:.4f}, μ = {mu:.6f}, Re = {Re:.1f}"

# --- Gradio Interface ---

demo = gr.Interface(
    fn=smart_simulate,
    inputs=[
        gr.Slider(16, 128, value=64, label="Grid Size (Nx)"),
        gr.Dropdown(["Density (ρ)", "Wall Velocity (U)", "Viscosity (μ)", "Reynolds Number (Re)"],
                    value="Viscosity (μ)", label="Select Variable to Compute"),

        gr.Number(value=1.0, label="Density (ρ)"),
        gr.Number(value=1.0, label="Wall Velocity (U)"),
        gr.Number(value=0.01, label="Viscosity (μ)"),
        gr.Number(value=100, label="Reynolds Number (Re)"),

        gr.Slider(0.0001, 0.01, value=0.001, step=0.0001, label="Time Step (dt)"),
        gr.Slider(1000, 50000, value=10000, step=1000, label="Max Iterations")
    ],
    outputs=[
        gr.Plot(label="U-Velocity Contour"),
        gr.Plot(label="Centerline Velocity"),
        gr.Plot(label="Streamlines"),
        gr.Plot(label="Velocity Divergence"),
        gr.Textbox(label="Computed Parameters")
    ],
    title="2D Lid Driven Cavity Flow Simulation with Smart Parameter Linking",
    description="Select 3 known quantities (ρ, U, μ, Re) and choose the 4th to compute dynamically. Ensures physical consistency."
)

demo.launch()



It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://bb829a7bd6eb63fd4d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
